## Project 

In this project we aim to develop an **estimate of velocity and position** from **GPS** data which is normally **noisy**.

Kalman filter offers a solution to the problem of processing the gps data and estimation of the state 

state of the vehicle can be defined as a vector $v_{x},v_{y},x,y$

## What do we need ? 

To implement kalman filter, we need two equations 

- Equation to describe the state transition behaviour, how is my state transitioning over time ?
- Model for the measurement process, how the measured data can be mapped to a state space !

state = $[x,y,v_{x},v_{y}]$

### State Transition 



 $$ state(t) = fn(state(t-1),control,noise) $$

 suppose that we replace state by x 

 $$ x(t) = fn(x(t-1),\epsilon) $$

to figure out the exact equation, let's try to resolve each term transion at it's own

$$v_{x}(t) = v_{x}(t-1)  + a_{x} \Delta t$$
$$v_{y}(t) = v_{y}(t-1)  + a_{y} \Delta t$$

$$ x(t) = x(t-1) + tv_{x}+a_{x}t^{2}$$

$$ y(t) = y(t-1) + tv_{y}+a_{y}t^{2}$$


### State Transition in Vector Form



# Project Implementation
## step 1 - Initializting the prediction 

In this project, we start first by state initialization using the stream operator `<<`, since we already know that the robot starts moving with 5 m/s at an angle of 45 degrees:

$$
\mathbf{state}_{t_0} =
\begin{bmatrix}
x \\
y \\
v_x \\
v_y
\end{bmatrix}
=
\begin{bmatrix}
0 \\
0 \\
5 \cos(45^\circ) \\
5 \sin(45^\circ)
\end{bmatrix}
$$

## Implementation in C++ (Eigen)

We use the `<<` stream operator to fill the vector in a quicker way:

```cpp
#include <Eigen/Dense>
#include <cmath>
using namespace Eigen;

void KalmanFilter::predictionStep(double dt){
    Vector4d state = Vector4d::Zero();
    state << 0, 0, 5.0*cos(M_PI/4), 5.0*sin(M_PI/4);

    return state;
}

```

### what is Eigen ? 
Eigen is a C++ template library for linear algebra, widely used for handling vectors, matrices, and related operations efficiently. It’s especially popular in robotics, physics simulations, computer graphics, and machine learning.

## Result  ? 

Just basic state initialization, if we build the simualtion without any modfication for this state over time, we will end up with growing estimation error which can be seen from the animation below.
<div style="text-align: center;">
    <img src="Figures/1.gif" width="400"/>
</div>


In summary we have a good initialization, but yet we don't update the predicted state properly as expected over time steps, an update step have to accomedate for the system and update correctly over time.


## Step 2 - Using a mathmatical model to predict motion over time.

lets assume that the mean of the state will progress over time throught the state transition matrix (stm)

in that cae : 
$$ state = state(t-1) * stm $$
$$
\mathbf{stm} =
\begin{bmatrix}
\overset{\raisebox{1em}{$x$}}{1} & 
\overset{\raisebox{1em}{$y$}}{0} & 
\overset{\raisebox{1em}{$v_x$}}{dt} & 
\overset{\raisebox{1em}{$v_y$}}{0} \\
0 & 1 & 0 & dt \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$


## Result  Step  -  2 

The vehicle have a more porper estimate of it's location over time, by using the mathmatical model to update each time step, resulting an overall zero error ! 


<div style="text-align: center;">
    <img src="Figures/2.gif" width="400"/>
</div>



